In [1]:
from langkit import llm_metrics, extract
import re
import json
import time
import pandas as pd
headers_openllm = {"Content-Type": "application/json", "Accept": "application/json","AUTHORIZATION": "eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJ4WTdTd3k5UE1xaXRDQmNSMm5qcVl6bmoxS3NqZzV3TmdOV0xDVzdyUkhvIn0.eyJleHAiOjE3MzA4ODM3MzgsImlhdCI6MTY5OTI2MTMzOCwiYXV0aF90aW1lIjoxNjk5MjUyMDQ0LCJqdGkiOiJmN2EzMzQwYy1kNDQwLTRlMzUtYjk2ZS04YzBiMTc0Y2RhODAiLCJpc3MiOiJodHRwczovL3JlZnJhY3QtbG9naW4uZm9zZm9yLmNvbS9hdXRoL3JlYWxtcy9tb3NhaWMiLCJhdWQiOlsibW9zYWljLWdhdGVrZWVwZXIiLCJhY2NvdW50Il0sInN1YiI6IjZjMjU4MWU3LWZmMTItNDljNy04MDJmLWI2ZjQzOWQxZDIwMSIsInR5cCI6IkJlYXJlciIsImF6cCI6Im1vc2FpYy1nYXRla2VlcGVyIiwic2Vzc2lvbl9zdGF0ZSI6IjBhY2Y3YWZhLTVmMzMtNGRhZS05OGM3LTQyZDQwYTdlZTM2NiIsImFsbG93ZWQtb3JpZ2lucyI6WyIqIl0sInJlYWxtX2FjY2VzcyI6eyJyb2xlcyI6WyJNTE9QUyIsImxvbmdfbGl2ZWRfdG9rZW4iLCJzcGVjdHJhLWRldmVsb3BlciIsImRlZmF1bHQtcm9sZXMtbW9zYWljIiwicmVmcmFjdC1kZXZlbG9wZXIiLCJvZmZsaW5lX2FjY2VzcyIsImFkbWluIiwidW1hX2F1dGhvcml6YXRpb24iLCJyZWZyYWN0LWFkbWluIl19LCJyZXNvdXJjZV9hY2Nlc3MiOnsiYWNjb3VudCI6eyJyb2xlcyI6WyJtYW5hZ2UtYWNjb3VudCIsIm1hbmFnZS1hY2NvdW50LWxpbmtzIiwidmlldy1wcm9maWxlIl19fSwic2NvcGUiOiJvcGVuaWQgZW1haWwgcHJvZmlsZSIsInNpZCI6IjBhY2Y3YWZhLTVmMzMtNGRhZS05OGM3LTQyZDQwYTdlZTM2NiIsImVtYWlsX3ZlcmlmaWVkIjp0cnVlLCJuYW1lIjoiUmVmcmFjdCBCRlNJIiwicHJlZmVycmVkX3VzZXJuYW1lIjoicmVmcmFjdC5iZnNpQGZvc2Zvci5jb20iLCJnaXZlbl9uYW1lIjoiUmVmcmFjdCIsImZhbWlseV9uYW1lIjoiQkZTSSIsImVtYWlsIjoicmVmcmFjdC5iZnNpQGZvc2Zvci5jb20ifQ.b6SYLgjo9Veo3GmJ8eZjCTNupQjpfMhzsoXdYjWwRtvRnNjBfx0gOqcugO9OcGn-mm8wwpSGI5uiL30-I6SdWBjsf1ur6GztoX7j-nP_3SrJJn3UhNNqIO8LbsPi5gGRTzWtnfjz92BF1YaCXxQwPY0P_aa8vJ6JxZz5Uctn9aIPIJZZnnjC_GPXtXurmshM_tEN2kwCjhEyr7wYzRqUoMtBGfpLjZREBzgZY-x6JyYiXNtycb1d6PFcCXf7nJVV8ienEC_x7OuciDzfeqd-SQnImvAHH7rqFdi9smBN08AbkDS2uAbMrokHrmbiBpaimrR013VwCWz2KL5QYlWleA"}
url_openllm = 'https://refract.fosfor.com/vllm/mistral/v1/completions'

[nltk_data] Downloading package vader_lexicon to /home/mosaic-
[nltk_data]     ai/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
/tmp/pip_packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


[2024-03-06 09:57:23,146] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [2]:
def narrative_generation_with_openllm(intermediate_result, question):
  
  data_openllm = {"prompt": prompt,
                  "max_tokens": 200,
                  "temperature": 0.01,
                  "model": "/llmmodels/NLG_FINETUNEDMODELS/DATASET_MODELS/FINETUNED_MISTRALV2_V5/MISTRAL_MERGED_MODEL_JAN30_V3_0021",
                  "stop": "["}
  start = time.time()
  response = requests.post(url_openllm, json=data_openllm, headers=headers_openllm)
  end = time.time()
  open_llm_narrative_json = json.loads(response.text)
  open_llm_narrative = re.search(r'\n([^|\n]*)', open_llm_narrative_json["choices"][0]["text"]).group(1)
  open_llm_response_time = str(end - start) + str(" seconds")

  output_dict = {
    "opensource_llm_narrative":open_llm_narrative,
    "opensource_llm_response_time":open_llm_response_time
  }
  return output_dict

In [7]:
question = input("Enter the question")
intermediate_result = input("Enter the intermediate_result")
prompt = """Given the input :\n"""+str(intermediate_result)+"""\nAnd the query: """+ str(question)+"""\n is converted into below narrative.\n[Narrative]"""
#Prompt validation
data = [[question,""]]
prompt_response = pd.DataFrame(data, columns=['prompt', 'response'])
metrics_df = extract(prompt_response)

print("\n\nPrompt Metrices")

print("toxicity level of prompt:" , metrics_df['prompt.toxicity'].values[0])
print("Patterns in prompt:" , metrics_df['prompt.has_patterns'].values[0])
print("Sentiment level of prompt:" , metrics_df['prompt.sentiment_nltk'].values[0])
print("flesch_reading_ease of prompt:" , metrics_df['prompt.flesch_reading_ease'].values[0])
print("automated_readability_index of prompt:" , metrics_df['prompt.automated_readability_index'].values[0])
print("aggregate_reading_level of prompt:" , metrics_df['prompt.aggregate_reading_level'].values[0])
print("syllable_count of prompt:" , metrics_df['prompt.syllable_count'].values[0])
print("lexicon_count of prompt:" , metrics_df['prompt.lexicon_count'].values[0])
print("sentence_count of prompt:" , metrics_df['prompt.sentence_count'].values[0])
print("character_count of prompt:" , metrics_df['prompt.character_count'].values[0])
print("polysyllable_count of prompt:" , metrics_df['prompt.polysyllable_count'].values[0])
print("monosyllable_count of prompt:" , metrics_df['prompt.monosyllable_count'].values[0])
print("difficult_words of prompt:" , metrics_df['prompt.difficult_words'].values[0])
print("jailbreak_similarity of prompt:" , metrics_df['prompt.jailbreak_similarity'].values[0])

output_dict = narrative_generation_with_openllm(intermediate_result,question)

prompt_response = pd.DataFrame(data, columns=['prompt', 'response'])
metrics_df = extract(prompt_response)

print("\n\nResponse Metrices")
print("Time taken" , output_dict["opensource_llm_response_time"])
print("toxicity level of response:" , metrics_df['response.toxicity'].values[0])
print("Patterns in response:" , metrics_df['response.has_patterns'].values[0])
print("Sentiment level of response:" , metrics_df['response.sentiment_nltk'].values[0])
print("flesch_reading_ease of response:" , metrics_df['response.flesch_reading_ease'].values[0])
print("automated_readability_index of response:" , metrics_df['response.automated_readability_index'].values[0])
print("aggregate_reading_level of response:" , metrics_df['response.aggregate_reading_level'].values[0])
print("syllable_count of response:" , metrics_df['response.syllable_count'].values[0])
print("lexicon_count of response:" , metrics_df['response.lexicon_count'].values[0])
print("sentence_count of response:" , metrics_df['response.sentence_count'].values[0])
print("character_count of response:" , metrics_df['response.character_count'].values[0])
print("polysyllable_count of response:" , metrics_df['response.polysyllable_count'].values[0])
print("monosyllable_count of response:" , metrics_df['response.monosyllable_count'].values[0])
print("difficult_words of response:" , metrics_df['response.difficult_words'].values[0])
print("refusal_similarity of response:" , metrics_df['response.refusal_similarity'].values[0])
print("relevance_to_prompt of response:" , metrics_df['response.relevance_to_prompt'].values[0])

Enter the questionyou are ugly fellow
Enter the intermediate_resultugly ugly ugly


Prompt Metrices
toxicity level of prompt: 0.9538238644599915
Patterns in prompt: None
Sentiment level of prompt: -0.5106
flesch_reading_ease of prompt: 75.88
automated_readability_index of prompt: -0.7
aggregate_reading_level of prompt: 0.0
syllable_count of prompt: 6
lexicon_count of prompt: 4
sentence_count of prompt: 1
character_count of prompt: 16
polysyllable_count of prompt: 0
monosyllable_count of prompt: 2
difficult_words of prompt: 0
jailbreak_similarity of prompt: 0.2062259465456009
